In [30]:
## Load the data

import urllib3
import pandas as pd
from io import StringIO

http = urllib3.PoolManager()

urls = ['https://raw.githubusercontent.com/uwescience/datasci_course_materials/master/assignment6/seattle_incidents_summer_2014.csv',
        'https://raw.githubusercontent.com/uwescience/datasci_course_materials/master/assignment6/sanfrancisco_incidents_summer_2014.csv',
        ]
cities = ['seattle', 'san_francisco']
data = {}
for city, url  in zip(cities,urls):
    r = http.request('GET', url)
    data[city] = r.data.decode('utf-8')
print(type(data['seattle']))
df_seattle      = pd.read_csv(StringIO(data['seattle']), sep=',')
df_san_franciso = pd.read_csv(StringIO(data['san_francisco']), sep=',')

print(df_seattle.head())
print('*'*50)
print(df_san_franciso.head())

#print(df_seattle.shape())

<class 'str'>
   RMS CDW ID  General Offense Number Offense Code  Offense Code Extension  \
0      483839              2015218538         2202                       0   
1      481252              2015213067         2610                       0   
2      481375              2015210301         2316                       0   
3      481690              2015209327         2599                       0   
4      478198              2015207880         2399                       3   

           Offense Type Summary Offense Code Summarized Offense Description  \
0    BURGLARY-FORCE-RES                 2200                       BURGLARY   
1  FRAUD-IDENTITY THEFT                 2600                          FRAUD   
2            THEFT-MAIL                 2300                     MAIL THEFT   
3           COUNTERFEIT                 2500                    COUNTERFEIT   
4             THEFT-OTH                 2300                 OTHER PROPERTY   

            Date Reported Occurred Date or

/home/dominik/anaconda3/envs/untitled/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
